Import the required modules

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import PIL
import PIL.Image
import json

In [ ]:
def load_angles_from_folder(directory):
    with open(directory,'r')  as ff:
        content = ff.readline()
        angles = content.split(sep='\t') #lines are separated by tabs
        angles = np.asarray(angles, dtype = np.float32) * (np.pi/180)
        return angles

In [ ]:
def load_images_from_folder(directory):
    images = []
    fileslist = os.listdir(directory)
    fileslist = sorted(fileslist , key = lambda fileslist: int(fileslist.split('.')[0]))
    #key=lambda x: int(x.split('_')[1]
    for filename in fileslist:
        #img = cv.imread(os.path.join(directory,filename))
        loc = os.path.join(directory,filename)
        img = tf.keras.preprocessing.image.load_img(loc, target_size=(300, 300), interpolation = 'nearest')
        img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        if img is not None:
            images.append(img)
    return np.asarray(images)

In [ ]:
directory_imgs = './Dataset/imgs/'
directory_angles = './Dataset/Labels.txt'
imgs = load_images_from_folder(directory_imgs)
angles = load_angles_from_folder(directory_angles)
plt.imshow(imgs[240])
print(angles[240])

Defining the Convolutional Neural Network

In [ ]:
base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top = False, input_shape = (300,300,3))
base_model.trainable = False
X = base_model.layers[-5].output
X = tf.keras.layers.Flatten()(X)
X = tf.keras.layers.Dense(1, 'linear', use_bias=True)(X)
model = tf.keras.Model(base_model.input, X)
opt = tf.keras.optimizers.Adam(learning_rate = 1e-6)
lossfunction = tf.keras.losses.MeanSquaredError()
model.summary()

Compile the model

In [ ]:
model.compile(optimizer = opt, loss = lossfunction, metrics = ['MeanAbsoluteError'])

Train and Save the model

In [ ]:
history = model.fit(imgs[:900],angles[:900], batch_size = 16,epochs = 150)

Save the model for test (Training takes time)

In [ ]:
model.save('./TrainedCNNs/for_test', save_format = 'h5' )

Store the history of training in a json file

In [ ]:
# Save the history to a JSON file
history_dict = history.history

# Specify the file path
file_path = './Training_History_Files/for_test.json'

# Convert the dictionary to a JSON string and save it to the file
with open(file_path, 'w') as json_file:
    json.dump(history_dict, json_file)

print(f"Training history saved to {file_path}")


Load the training results

In [ ]:
file_path = './Training_History_Files/for_test.json'
history_dict = json.load(open(file_path, 'r'))

Plot the training curves

In [ ]:
font = {'weight' : 'bold',
        'size'   : 12}
plt.rc ('font', **font)


figure1 = plt.figure(figsize=(5,5))
RootMeanSquareError =np.array(history_dict['loss'])
SampleNo = np.arange(1,len(RootMeanSquareError) + 1)



plt.plot(SampleNo,RootMeanSquareError , color = 'blue')
plt.xlabel('Epochs' , fontsize = 12 , weight='bold')
plt.ylabel(r'MSE (rad$^2$)' , fontsize = 12 , weight='bold')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.savefig('./Results_Paper/mse_for_test.eps', format='eps')
figure2 = plt.figure(figsize=(5,5))
MeanAbsError = history_dict['mean_absolute_error']


plt.plot(SampleNo,MeanAbsError , color = 'blue')
plt.xlabel('Epochs' , fontsize = 12 , weight='bold')
plt.ylabel('MAE (rad)' , fontsize = 12 , weight='bold')
plt.savefig('./Results_Paper/mae_for_test.eps', format='eps')